In [1]:
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
from address_convertor import pubkey_to_address, get_pubkey

In [2]:
print(rpc_user, rpc_password)

dnlab dnlab2020


In [5]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('/home/dnlab/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
print(pid_path)
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is running!')

/home/dnlab/.bitcoin/bitcoind.pid
Bitcoind PID: 15125



In [7]:
# preparation database
import sqlite3

db_path = 'cluster.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

    
def create_txhash_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxHash (
                     txhash TEXT PRIMARY KEY,
                     height INTEGER NOT NULL);
                ''')

def create_txin_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxIn (
                     address TEXT NOT NULL,
                     txhash TEXT NOT NULL);
                ''')

def create_txout_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxOut (
                     address TEXT NOT NULL,
                     txhash TEXT NOT NULL);
                ''')    

def create_cluster_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Cluster (
                     address TEXT PRIMARY KEY,
                     number INTEGER NOT NULL);
                ''')

    
def insert_txhash(txhash, height):
    cur.execute('''INSERT OR REPLACE INTO TxHash (
                       txhash, height) VALUES (
                       ?, ?);
                    ''', (txhash, height))
    
def insert_txin(address, txhash):
    cur.execute('''INSERT OR REPLACE INTO TxIn (
                       address, txhash) VALUES (
                       ?, ?);
                    ''', (address, txhash))

def insert_txout(address, txhash):
    cur.execute('''INSERT OR REPLACE INTO TxOut (
                       address, txhash) VALUES (
                       ?, ?);
                    ''', (address, txhash))

def insert_cluster(address, number):
    cur.execute('''INSERT OR IGNORE INTO Cluster (
                       address, number) VALUES (
                       ?, ?);
                    ''', (address, number))

def do_cluster(addresses, number):
    cur.executemany(f'''UPDATE Cluster SET number = {number} WHERE address IN ({",".join(addresses)})''')

    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    


def get_max_height():
    cur.execute('''SELECT MAX(height) FROM TxHash;''')
    return cur.fetchone()[0]

def get_cluster_number(addresses):
    cur.execute(f'''SELECT MIN(number) FROM Cluster WHERE address IN ({",".join(addresses)})''')
    return cur.fetchone()[0]

In [8]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

Best Block Heights: 629427, Time: 2020-05-08T09:51:16+09:00


In [9]:

%%time

create_txin_table()
create_txout_table()
create_cluster_table()
create_txhash_table()

CPU times: user 1.03 ms, sys: 192 µs, total: 1.22 ms
Wall time: 518 µs


In [10]:
import time
import pickle

if os.path.exists('taking.pickle'):
    with open('taking.pickle', 'rb') as f:
        taking = pickle.load(f)
else:
    taking = list()

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

start_height = get_max_height()
if start_height is None:
    start_height = 0
end_height = best_block['height']-99 # default
end_height = 200000
print(f'Start from {start_height}')
 

stime = time.time()
for height in range(start_height, end_height):
    begin_transactions()
    block_hash = rpc_connection.getblockhash(height)
    block = rpc_connection.getblock(block_hash, 2)
    for tx in block['tx']:
        insert_txhash(tx['hash'], height)
        iaddresses = set()
        oaddresses = set()
        for vin in tx['vin']:
            if 'coinbase' in vin.keys():
                continue
            ptx = rpc_connection.getrawtransaction(vin['txid'], 1)
            pvout = ptx['vout'][vin['vout']]
            if pvout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                 'witness_v0_keyhash', 'witness_v0_scripthash',
                                                 'multisig'):
                iaddresses = pvout['scriptPubKey']['addresses']
            elif pvout['scriptPubKey']['type'] == 'pubkey':
                iaddresses = [pubkey_to_address(get_pubkey(pvout['scriptPubKey']['hex']))]    
            iaddresses = set(iaddresses)
            for address in iaddresses:
                insert_txin(address, tx['hash'])
        for vout in tx['vout']:
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'multisig'):
                oaddresses = vout['scriptPubKey']['addresses']
            elif vout['scriptPubKey']['type'] == 'pubkey':
                oaddresses = [pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))]
            oaddresses = set(oaddresses)
            for address in oaddresses:
                insert_txout(address, tx['hash'])
                insert_cluster(address, -1)
    commit_transactions()
    if height % 1000 == 0:
        tt = int(time.time()-stime)
        print(f'[{tt}] {height} Done')
        taking.append(tt)
        stime = time.time()
tt = int(time.time()-stime)
print(f'[{tt}] {height} Done')
taking.append(tt)

Start from 2559
[4] 3000 Done
[10] 4000 Done
[10] 5000 Done
[10] 6000 Done
[10] 7000 Done
[10] 8000 Done
[10] 9000 Done
[10] 10000 Done
[10] 11000 Done
[10] 12000 Done
[11] 13000 Done
[10] 14000 Done
[10] 15000 Done
[10] 16000 Done
[10] 17000 Done
[11] 18000 Done
[14] 19000 Done
[15] 20000 Done
[17] 21000 Done
[18] 22000 Done
[18] 23000 Done
[18] 24000 Done
[18] 25000 Done
[19] 26000 Done
[19] 27000 Done
[18] 28000 Done
[18] 29000 Done
[19] 30000 Done
[18] 31000 Done
[18] 32000 Done
[18] 33000 Done
[18] 34000 Done
[18] 35000 Done
[18] 36000 Done
[18] 37000 Done
[18] 38000 Done
[18] 39000 Done
[20] 40000 Done
[19] 41000 Done
[19] 42000 Done
[19] 43000 Done
[20] 44000 Done
[20] 45000 Done
[20] 46000 Done
[19] 47000 Done
[19] 48000 Done
[18] 49000 Done
[21] 50000 Done
[20] 51000 Done
[28] 52000 Done
[21] 53000 Done
[20] 54000 Done
[22] 55000 Done
[23] 56000 Done
[19] 57000 Done
[20] 58000 Done
[19] 59000 Done
[20] 60000 Done
[20] 61000 Done
[19] 62000 Done
[19] 63000 Done
[23] 64000 Done


In [ ]:
conn.close()

In [ ]:
import pickle

with open('taking.pickle', 'wb') as f:
    pickle.dump(taking, f)